In [355]:
#%load_ext lab_black

In [356]:
## Importing all needed libraries and modules
## Setting up BigQuery access credentials for SQL parsing of reporting data
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandas as pd
import numpy as np
import os
import warnings
import datetime

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

credentials = service_account.Credentials.from_service_account_file(
    "/Users/miguelcouto/Downloads/zattoo-dataeng-e5f45785174f.json"
)

project_id = "zattoo-dataeng"
client = bigquery.Client(credentials=credentials, project=project_id)

In [357]:
## Prepare dataframe
df = pd.read_csv('/Users/miguelcouto/PycharmProjects/pypayment_v2/raw/google/PlayApps_202203.csv')

In [358]:
## Renaming and reordering the dataframe so it keeps consistency with the reports from the other app shops
reporting_df = df[
    ['Description', 'Transaction Date', 'Transaction Time', 'Tax Type', 'Transaction Type',
     'Refund Type', 'Product Title', 'Sku Id', 'Buyer Country',
     'Buyer Currency', 'Amount (Buyer Currency)', 'Currency Conversion Rate',
     'Merchant Currency', 'Amount (Merchant Currency)', 'Offer ID']]

reporting_df.rename({'Description': 'description',
                     'Transaction Time': 'transaction_time',
                     'Transaction Date': 'transaction_date',
                     'Transaction Type': 'type_of_transaction',
                     'Tax Type': 'tax_type',
                     'Refund Type': 'refund_type',
                     'Product Title': 'product_title',
                     'Sku Id': 'vendor_sku',
                     'Buyer Country': 'buyer_country_code',
                     'Buyer Currency': 'buyer_currency',
                     'Merchant Currency': 'merchant_currency',
                     'Currency Conversion Rate': 'currency_conversion_rate',
                     'Amount (Buyer Currency)': 'payout_buyer_currency',
                     'Amount (Merchant Currency)': 'payout_chf',
                     'Offer ID': 'offer_id',
                     },
                    axis=1, inplace=True)

## Converting date columns to datetime and getting rid of unnecessary columns
reporting_df['transaction_date'] = pd.to_datetime(df['Transaction Date'])
reporting_df['transaction_time'] = pd.to_timedelta(df['Transaction Time'])

reporting_df['transaction_date'] = pd.to_datetime(reporting_df['transaction_date'] + reporting_df['transaction_time'],
                                                  format='%m-%d-%Y%H:%M:%S')

reporting_df.drop('transaction_time', inplace=True, axis=1)

In [359]:
## We only report DE, CH, AT. Everything except DACH goes to CH attributed abroad in the column domestic_abroad.
reporting_df['domestic_abroad'] = pd.np.where(
    reporting_df['buyer_country_code'].str.lower() == 'de', 'domestic',
    pd.np.where(reporting_df['buyer_country_code'].str.lower() == 'at', 'domestic',
                pd.np.where(reporting_df['buyer_country_code'].str.lower() == 'ch', 'domestic', 'abroad')))

reporting_df['country_code_clean'] = pd.np.where(
    reporting_df['buyer_country_code'].str.lower() == 'de', 'DE',
    pd.np.where(reporting_df['buyer_country_code'].str.lower() == 'at', 'AT',
                pd.np.where(reporting_df['buyer_country_code'].str.lower() == 'ch', 'CH', 'CH')))

In [360]:
# ## Build join to dim.countries and get country_names
# dim_countries = """
# select distinct countrycode as country_code,
#                 name as country_name
# from dim.countries
# ;
# """
#
# df_countries = pandas_gbq.read_gbq(dim_countries, project_id=project_id, progress_bar_type=None)
#
# reporting_df = reporting_df.merge(df_countries, how='left', left_on='country_code_clean', right_on='country_code')

In [361]:
## Finance Subs reporting will happen once at end of each month and for that the reporting_month will be defined based on the mode of the transaction_date.
reporting_df['reporting_month'] = reporting_df['transaction_date'].dt.to_period('M').dt.strftime('%Y-%m')
reporting_month = reporting_df.reporting_month.mode()
reporting_df['reporting_month'] = reporting_df['reporting_month'].apply(lambda x: reporting_month)

In [362]:
## Adding payment_method column
reporting_df['payment_method'] = 'google'

In [363]:
## SQL query to enrich the dataframe with more specific product data (class, detailed class, length)
skus_list = str(set(reporting_df['vendor_sku'].to_list()))

skus_expand = f"""select distinct rlv.SKU as sku,
                rlv.product_class,
                rlv.detailed_product_class,
                rlv.product_length
from b2c_middleware.reporting_layer_view rlv
where true
  and rlv.SKU in ({skus_list[1:-1]})
  and rlv.app_shop_id = 'google'
  and rlv.product_class is not null
  and rlv.detailed_product_class is not null
  ;"""

df_skus_expand = pandas_gbq.read_gbq(skus_expand, project_id=project_id, progress_bar_type=None)

reporting_df = reporting_df.merge(df_skus_expand, how='left',
                                  left_on='vendor_sku', right_on='sku').drop('vendor_sku', axis=1)

In [364]:
## Adding units column
reporting_df['units'] = pd.np.where(
    reporting_df['type_of_transaction'].str.lower() == 'charge', 1,
    pd.np.where(reporting_df['type_of_transaction'].str.lower() == 'charge refund', -1, 0))

In [365]:
## Creating product_length_months column
reporting_df['product_length_months'] = reporting_df['product_length']

reporting_df['product_length_months'] = reporting_df['product_length'].replace(
    {31: 1, 92: 3, 365: 12})

In [366]:
## Enriching google with data from our datamarts which already has a lot of clean business logic
trxs_list = str(set(reporting_df['description'].to_list()))

trxs_expand = f"""
select rlv.transaction_id,
       rlv.term_end,
       pav.latest_receipt_orderid,
       rlv.creation_time,
       rlv.store_fees_applicable_percentage as store_fees_percentage,
       rlv.vat_percentage,
--        pe_eur.exchange_rate                 as exchange_rate_eur_to_chf,
--        rlv.vat_eur,
--        rlv.vat_chf,
--        rlv.new_booking_net_chf,
--        rlv.new_booking_net_EUR,
--        rlv.renewal_booking_net_CHF,
--        rlv.renewal_booking_net_EUR,
       rlv.is_renewal
from b2c_middleware.middlelayer_androidtransaction rlv
         left join b2c_middleware_import.payment_androidtransaction_view pav
                   on pav.id = rlv.transaction_id
         LEFT JOIN b2c_middleware_import.payment_exchangerate pe
                   ON pe.from_currency = rlv.currency
                       AND DATE_TRUNC(pe.day, MONTH) =
                           DATE_TRUNC(DATE(rlv.creation_time), MONTH)
         LEFT JOIN b2c_middleware_import.payment_exchangerate pe_eur
                   ON pe_eur.from_currency = 'EUR'
                       AND DATE_TRUNC(pe_eur.day, MONTH) =
                           DATE_TRUNC(DATE(rlv.creation_time), MONTH)
where true
  AND pe.day >= DATE('2008-08-01')
  AND pe_eur.day >= DATE('2008-08-01')
  and pav.latest_receipt_orderid in ({trxs_list[1:-1]})
"""

df_trxs_expand = pandas_gbq.read_gbq(trxs_expand, project_id=project_id, progress_bar_type=None)

reporting_df = reporting_df.merge(df_trxs_expand, how='left',
                                  left_on='description', right_on='latest_receipt_orderid').drop(
    'latest_receipt_orderid', axis=1)

reporting_df.drop_duplicates(subset=["description", 'type_of_transaction'], inplace=True)

In [367]:
## Lowering caps for uniformity
reporting_df.rename({'new_booking_net_EUR': 'new_booking_net_eur',
                     'renewal_booking_net_CHF': 'renewal_booking_net_chf',
                     'renewal_booking_net_EUR': 'renewal_booking_net_eur',
                     },
                    axis=1, inplace=True)

In [368]:
## 12M subs in DE and AT should be allocated to CH + abroad
reporting_df['domestic_abroad'][(reporting_df['product_length_months'] == 12) & (
    reporting_df['buyer_country_code'].str.lower().isin(['de', 'at']))] = 'abroad'

reporting_df['country_code_clean'][
    (reporting_df['product_length_months'] == 12) & (
        reporting_df['buyer_country_code'].str.lower().isin(['de', 'at']))] = 'CH'

In [369]:
## Build join to dim.countries and get country_names
dim_countries = """
select distinct countrycode as country_code,
                name as country_name
from dim.countries
;
"""

df_countries = pandas_gbq.read_gbq(dim_countries, project_id=project_id, progress_bar_type=None)

reporting_df = reporting_df.merge(df_countries, how='left', left_on='country_code_clean', right_on='country_code')

In [370]:
## Calculating VAT from google instead of getting them from our datamarts because google has its own exchange rates
df_vat_chf = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(['tax', 'tax refund']))].groupby('transaction_id')[
    'payout_chf'].sum().reset_index()

df_vat_chf.rename({'payout_chf': 'vat_chf'
                   },
                  axis=1, inplace=True)

reporting_df = reporting_df.merge(df_vat_chf, how='left', left_on='transaction_id', right_on='transaction_id')

df_vat_buyer_currency = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(['tax', 'tax refund']))].groupby('transaction_id')[
    'payout_buyer_currency'].sum().reset_index()

df_vat_buyer_currency.rename({'payout_buyer_currency': 'vat_buyer_currency'
                              },
                             axis=1, inplace=True)

reporting_df = reporting_df.merge(df_vat_buyer_currency, how='left', left_on='transaction_id',
                                  right_on='transaction_id')

In [371]:
## Calculating google fee refund
df_fee_chf = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(['google fee refund', 'google fee']))].groupby(
    'transaction_id')['payout_chf'].sum().reset_index()

df_fee_chf.rename({'payout_chf': 'fee_chf'
                   },
                  axis=1, inplace=True)

reporting_df = reporting_df.merge(df_fee_chf, how='left', left_on='transaction_id', right_on='transaction_id')

df_fee_buyer_currency = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(['google fee refund', 'google fee']))].groupby(
    'transaction_id')['payout_buyer_currency'].sum().reset_index()

df_fee_buyer_currency.rename({'payout_buyer_currency': 'fee_buyer_currency'
                              },
                             axis=1, inplace=True)

reporting_df = reporting_df.merge(df_fee_buyer_currency, how='left', left_on='transaction_id',
                                  right_on='transaction_id')

In [372]:
## Calculating net bookings from google instead of getting them from our datamarts because google has its own exchange rates
df_net_booking_chf = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(
    ['google fee refund', 'google fee', 'charge', 'charge refund']))].groupby('transaction_id')[
    'payout_chf'].sum().reset_index()

df_net_booking_chf.rename({'payout_chf': 'net_booking_chf'
                           },
                          axis=1, inplace=True)

reporting_df = reporting_df.merge(df_net_booking_chf, how='left', left_on='transaction_id', right_on='transaction_id')

df_net_booking_buyer_currency = reporting_df[(reporting_df['type_of_transaction'].str.lower().isin(
    ['google fee refund', 'google fee', 'charge', 'charge refund']))].groupby('transaction_id')[
    'payout_buyer_currency'].sum().reset_index()

df_net_booking_buyer_currency.rename({'payout_buyer_currency': 'net_booking_buyer_currency'
                                      },
                                     axis=1, inplace=True)

reporting_df = reporting_df.merge(df_net_booking_buyer_currency, how='left', left_on='transaction_id',
                                  right_on='transaction_id')

In [373]:
## Calculating sales price from google instead of getting them from our datamarts because google has its own exchange rates
reporting_df['sales_price_chf'] = reporting_df['payout_chf'] - reporting_df['fee_chf']

reporting_df['sales_price_buyer_currency'] = reporting_df['payout_buyer_currency'] - reporting_df['fee_buyer_currency']

In [374]:
## Calculating charge from google instead of getting them from our datamarts because google has its own exchange rates
reporting_df['charge_chf'] = reporting_df['sales_price_chf'] - reporting_df['vat_chf']

reporting_df['charge_buyer_currency'] = reporting_df['sales_price_buyer_currency'] - reporting_df['vat_buyer_currency']

In [375]:
## avg_price_sales_per_sub calculation
reporting_df['avg_price_sales_per_sub_chf'] = reporting_df['sales_price_chf'] / reporting_df['units']

reporting_df['avg_price_sales_per_sub_buyer_currency'] = reporting_df['sales_price_buyer_currency'] / reporting_df['units']

In [376]:
## Removing UTC timezone from term_end
reporting_df['term_end'] = pd.to_datetime(reporting_df['term_end'])

reporting_df['term_end'] = reporting_df['term_end'].dt.strftime('%Y-%m-%d %H:%M:%S')

reporting_df['term_end'] = pd.to_datetime(reporting_df['term_end'])

In [377]:
## Adding artificially created term_end_date based on initial transaction_date
reporting_df['term_end'][reporting_df['term_end'].isna()] = reporting_df['transaction_date'] + reporting_df['product_length'].astype('timedelta64[ns]')

In [378]:
## Calculate product_term_length_months
reporting_df["product_term_length_months"] = (
        (reporting_df["term_end"].dt.year - reporting_df["transaction_date"].dt.year) * 12
        + (reporting_df["term_end"].dt.month - reporting_df["transaction_date"].dt.month)
        + 1
)

## A bit of housekeeping
reporting_df["product_term_length_months"][reporting_df["product_term_length_months"] < 0] = 0

In [379]:
## Replacing product_term_length_months for exceptions where value is 2 instead of 1
shorter_subs = (reporting_df['term_end'] - reporting_df['transaction_date']).dt.days <= 30
shorter_subs_replacer = reporting_df[shorter_subs][
    (reporting_df["product_term_length_months"] == 2) & ((reporting_df['term_end']).dt.day == 1)].index.to_list()
reporting_df["product_term_length_months"].loc[shorter_subs_replacer] = 1

In [380]:
## Cleaning up a bit
reporting_df.replace(np.inf, 0, inplace=True)
reporting_df.replace(-np.inf, 0, inplace=True)

In [381]:
## Adding revenue_month_number - this will help in further calculations
reporting_df = reporting_df.loc[reporting_df.index.repeat(reporting_df['product_term_length_months'])].reset_index(
    drop=True)
reporting_df['revenue_month_number'] = 1
reporting_df['revenue_month_number'] = reporting_df.groupby(["transaction_id", 'type_of_transaction'])[
    'revenue_month_number'].cumsum()

In [382]:
## Adding max_month_date to tackle the specifications of active_sub_month_end
reporting_df['max_month_date'] = reporting_df.groupby([reporting_df['term_end'].dt.to_period('M'), 'type_of_transaction'])[
    'term_end'].transform('max')
reporting_df['max_month_date'] = reporting_df['max_month_date'].dt.normalize() + pd.Timedelta('23:59:59')

In [383]:
## Get last indices of each transaction_id group
last_idxs_charges = (
        len(reporting_df[reporting_df['type_of_transaction'] == 'charge'])
        - np.unique(
    reporting_df['transaction_id'][reporting_df['type_of_transaction'] == 'charge'].values[::-1],
    return_index=1,
)[1]
        - 1
)

## Add revenue_month_date
reporting_df["revenue_month_date"] = reporting_df["transaction_date"].to_numpy().astype("datetime64[M]")
reporting_df["revenue_month_date"][reporting_df["type_of_transaction"] == "refund"] = (
    reporting_df["transaction_date"].to_numpy().astype("datetime64[M]")
)

reporting_df["revenue_month_date"] = reporting_df.apply(
    lambda x: x["revenue_month_date"]
              + pd.offsets.MonthEnd(x["revenue_month_number"])
              + pd.offsets.MonthBegin(-1),
    axis=1,
)

reporting_df["product_term_length"] = reporting_df["revenue_month_date"].apply(
    lambda t: pd.Period(t, freq="S").days_in_month
)

reporting_df["product_term_length"][reporting_df["transaction_date"] > reporting_df["revenue_month_date"]] = (
                                                                                                                     reporting_df[
                                                                                                                         "transaction_date"].dt.daysinmonth -
                                                                                                                     reporting_df[
                                                                                                                         "transaction_date"].dt.day
                                                                                                             ) + 1

## Fix last position of product_term_length per transaction_id for charges
reporting_df["product_term_length"].iloc[last_idxs_charges] = (
        reporting_df["term_end"].iloc[last_idxs_charges] -
        reporting_df["revenue_month_date"].iloc[last_idxs_charges]
).dt.days

reporting_df = reporting_df[reporting_df['product_term_length'] > 0]

last_idxs = (
        len(reporting_df)
        - np.unique(
    reporting_df['transaction_id'].values[::-1],
    return_index=1,
)[1]
        - 1
)

reporting_df["product_term_length"].iloc[last_idxs] = (
        reporting_df["term_end"].iloc[last_idxs] -
        reporting_df["revenue_month_date"].iloc[last_idxs]
).dt.days

In [384]:
# reporting_df["active_sub_month_end"] = 1
#
# reporting_df["active_sub_month_end"].iloc[last_idxs] = 0
#
# reporting_df["active_sub_month_end"][
#     (reporting_df["revenue_month_number"] == reporting_df['product_term_length_months']) & (
#             reporting_df["term_end"] < reporting_df['max_month_date'])
#     ] = 0

## Set active_sub_month_end = 0 by default
reporting_df['active_sub_month_end'] = 0
reporting_df['active_sub_month_end'][
    (reporting_df['term_end'] > (reporting_df['revenue_month_date'] + pd.offsets.MonthBegin(1)))] = 1
reporting_df["active_sub_month_end"][reporting_df["type_of_transaction"] == "trial"] = 0

## mark all one transaction subscriptions as 1
# reporting_df["active_sub_month_end"][
#     (reporting_df["revenue_month_number"] <= 1) & (reporting_df["product_term_length_months"] <= 1)
#     ] = 1

## Mark all refund transactions as 'active_sub_month_end'] * -1
reporting_df["active_sub_month_end"][reporting_df["type_of_transaction"] != "charge"] = reporting_df[
                                                                                            'active_sub_month_end'] * -1

reporting_df.reset_index(drop=True, inplace=True)

In [385]:
## active_sub_content follows the same logic as active_sub_month_end except it doesn't count the last month
reporting_df["active_sub_content"] = reporting_df.active_sub_month_end

reporting_df["active_sub_content"][(reporting_df["revenue_month_number"] > reporting_df['product_length_months'])] = 0

reporting_df["active_sub_content"][reporting_df["type_of_transaction"] == "trial"] = 0

## Total_days of product_term_length per transaction_id
reporting_df["total_days"] = reporting_df.groupby(["transaction_id", 'type_of_transaction'])[
    "product_term_length"].transform("sum")

In [386]:
## Adding product_term_length_months
reporting_df['product_term_length_months'] = reporting_df.groupby(["transaction_id", 'type_of_transaction'])[
    'revenue_month_number'].transform('max')

In [387]:
## Calculate total_revenue_net fields
reporting_df["total_revenue_net_chf"] = (
        reporting_df["net_booking_chf"] / reporting_df["total_days"] * reporting_df["product_term_length"]
)

reporting_df["total_revenue_net_buyer_currency"] = (
        reporting_df["net_booking_buyer_currency"] / reporting_df["total_days"] * reporting_df["product_term_length"]
)

In [388]:
## Remove values from all lines of cols group except first
one_line_cols = ["vat_eur", 'vat_chf', 'payout_eur', 'payout_chf', 'sales_price_eur', 'sales_price_chf', 'fee_eur',
                 'fee_chf', 'new_booking_net_eur',
                 'new_booking_net_chf', 'renewal_booking_net_eur',
                 'renewal_booking_net_chf', 'store_fees_eur', 'store_fees_chf',
                 'charge_eur', 'charge_chf',
                 'vat_percentage',
                 'avg_price_sales_per_sub',
                 'units'
                 ]

for col in one_line_cols:
    reporting_df.loc[
        reporting_df["revenue_month_number"] > 1,
        [col],
    ] = 0.0

In [389]:
## Standardizing the report
reporting_df[['product_group_finance']] = ''

reporting_df[['currency', 'subscription_status']] = np.nan

reporting_df[
    ['store_fees_eur', 'store_fees_percentage', 'store_fees_percentage', 'store_fees_chf', 'renewal_booking_net_eur',
     'renewal_booking_net_chf', 'exchange_rate_eur_to_chf', 'total_revenue_net_eur']] = 0

In [390]:
## Reorder dataframe
reporting_df = reporting_df[
    [
        'transaction_id',
        'transaction_date',
        'term_end',
        'reporting_month',
        'country_name',
        'country_code',
        'currency',
        'sku',
        'subscription_status',
        'type_of_transaction',
        'payment_method',
        'product_class',
        'product_group_finance',
        'product_length',
        'product_length_months',
        'product_term_length',
        'domestic_abroad',
        'vat_percentage',
        'exchange_rate_eur_to_chf',
        'store_fees_eur',
        'store_fees_chf',
        'store_fees_percentage',
        'units',
        'charge_eur',
        'sales_price_eur',
        'fee_eur',
        'vat_eur',
        'new_booking_net_eur',
        'renewal_booking_net_eur',
        'payout_eur',
        'total_revenue_net_eur',
        'charge_chf',
        'sales_price_chf',
        'fee_chf',
        'vat_chf',
        'new_booking_net_chf',
        'renewal_booking_net_chf',
        'payout_chf',
        'total_revenue_net_chf',
        'revenue_month_number',
        'revenue_month_date',
        'active_sub_month_end',
        'active_sub_content'
    ]
]

## Define BQ table schema
bq_schema = [
    {"name": "transaction_id", "type": "STRING"},
    {"name": "transaction_date", "type": "TIMESTAMP"},
    {"name": "term_end", "type": "TIMESTAMP"},
    {"name": "reporting_month", "type": "STRING"},
    {"name": "country_name", "type": "STRING"},
    {"name": "country_code", "type": "STRING"},
    {"name": "currency", "type": "STRING"},
    {"name": "sku", "type": "STRING"},
    {"name": "subscription_status", "type": "STRING"},
    {"name": "type_of_transaction", "type": "STRING"},
    {"name": "payment_method", "type": "STRING"},
    {"name": "product_class", "type": "STRING"},
    {"name": "product_group_finance", "type": "STRING"},
    {"name": "product_length", "type": "INTEGER"},
    {"name": "product_length_months", "type": "INTEGER"},
    {"name": "product_term_length", "type": "INTEGER"},
    {"name": "domestic_abroad", "type": "STRING"},
    {"name": "vat_percentage", "type": "FLOAT"},
    {"name": "exchange_rate_eur_to_chf", "type": "FLOAT"},
    {"name": "store_fees_eur", "type": "INTEGER"},
    {"name": "store_fees_chf", "type": "INTEGER"},
    {"name": "store_fees_percentage", "type": "INTEGER"},
    {"name": "units", "type": "INTEGER"},
    {"name": "charge_eur", "type": "FLOAT"},
    {"name": "sales_price_eur", "type": "FLOAT"},
    {"name": "fee_eur", "type": "FLOAT"},
    {"name": "vat_eur", "type": "FLOAT"},
    {"name": "new_booking_net_eur", "type": "FLOAT"},
    {"name": "renewal_booking_net_eur", "type": "FLOAT"},
    {"name": "payout_eur", "type": "FLOAT"},
    {"name": "total_revenue_net_eur", "type": "FLOAT"},
    {"name": "charge_chf", "type": "FLOAT"},
    {"name": "sales_price_chf", "type": "FLOAT"},
    {"name": "fee_chf", "type": "FLOAT"},
    {"name": "vat_chf", "type": "FLOAT"},
    {"name": "new_booking_net_chf", "type": "FLOAT"},
    {"name": "renewal_booking_net_chf", "type": "FLOAT"},
    {"name": "payout_chf", "type": "FLOAT"},
    {"name": "total_revenue_net_chf", "type": "FLOAT"},
    {"name": "revenue_month_number", "type": "INTEGER"},
    {"name": "revenue_month_date", "type": "STRING"},
    {"name": "active_sub_month_end", "type": "INTEGER"},
    {"name": "active_sub_content", "type": "INTEGER"}
]

In [393]:
reporting_df

,transaction_id,transaction_date,term_end,reporting_month,country_name,country_code,currency,sku,subscription_status,type_of_transaction,payment_method,product_class,product_group_finance,product_length,product_length_months,product_term_length,domestic_abroad,vat_percentage,exchange_rate_eur_to_chf,store_fees_eur,store_fees_chf,store_fees_percentage,units,charge_eur,sales_price_eur,fee_eur,vat_eur,new_booking_net_eur,renewal_booking_net_eur,payout_eur,total_revenue_net_eur,charge_chf,sales_price_chf,fee_chf,vat_chf,new_booking_net_chf,renewal_booking_net_chf,payout_chf,total_revenue_net_chf,revenue_month_number,revenue_month_date,active_sub_month_end,active_sub_content
0,36122802.0,2022-03-01 12:00:15,2022-03-28 10:59:53,2022-03,Germany,DE,NaN,com.zattoo.android.de.1399.10.freetrial,NaN,Charge,google,ultimate,,31,1,27,domestic,19.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10.28,10.280000,1,2022-03-01,0,0
1,36122767.0,2022-03-01 12:01:17,2022-04-01 11:01:04,2022-03,Germany,DE,NaN,com.zattoo.android.de.999.10.freetrial,NaN,Charge,google,premium,,31,1,31,domestic,19.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,7.33,7.330000,1,2022-03-01,-1,-1
2,36122769.0,2022-03-01 12:01:49,2022-04-01 11:01:29,2022-03,Germany,DE,NaN,com.zattoo.android.de.999.10.freetrial,NaN,Charge,google,premium,,31,1,31,domestic,19.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,7.33,7.330000,1,2022-03-01,-1,-1
3,36122753.0,2022-03-01 12:05:50,2022-04-01 11:05:39,2022-03,Switzerland,CH,NaN,com.zattoo.android.ch.2000.10.freetrial,NaN,Charge,google,ultimate,,31,1,31,domestic,7.7,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,19.93,21.36,-2.79,1.43,NaN,0,18.57,15.780000,1,2022-03-01,-1,-1
4,36122753.0,2022-03-01 12:05:50,2022-04-01 11:05:39,2022-03,Switzerland,CH,NaN,com.zattoo.android.ch.2000.10.freetrial,NaN,Google fee,google,ultimate,,31,1,31,domestic,7.7,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0,-1.43,0.00,-2.79,1.43,NaN,0,-2.79,15.780000,1,2022-03-01,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73570,36479792.0,2022-03-31 11:59:39,2022-05-01 09:59:28,2022-03,Germany,DE,NaN,com.zattoo.android.de.999.10.freetrial,NaN,Charge,google,premium,,31,1,30,domestic,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00,0.0,0,0.00,7.064516,2,2022-04-01,-1,0
73571,36479792.0,2022-03-31 11:59:39,2022-05-01 09:59:28,2022-03,Germany,DE,NaN,com.zattoo.android.de.999.10.freetrial,NaN,Charge,google,premium,,31,1,0,domestic,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00,0.0,0,0.00,0.000000,3,2022-05-01,0,0
73572,36479789.0,2022-03-31 11:59:42,2022-05-01 09:59:33,2022-03,Germany,DE,NaN,com.zattoo.android.de.1399.10.freetrial,NaN,Charge,google,ultimate,,31,1,1,domestic,19.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10.24,0.330323,1,2022-03-01,-1,-1
73573,36479789.0,2022-03-31 11:59:42,2022-05-01 09:59:33,2022-03,Germany,DE,NaN,com.zattoo.android.de.1399.10.freetrial,NaN,Charge,google,ultimate,,31,1,30,domestic,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.00,0.00,0.00,0.00,0.0,0,0.00,9.909677,2,2022-04-01,-1,0


In [394]:
## Export to csv
path = r"/Users/miguelcouto/Desktop/"

reporting_df.to_csv(os.path.join(path, fr'subs_reporting_google{reporting_month[0].replace("-", "_")}.csv'))

In [395]:
## Export to BQ table
pandas_gbq.to_gbq(
    dataframe=reporting_df,
    destination_table=f"finance.subs_reporting_google{reporting_month[0].replace('-', '')}",
    project_id="zattoo-dataeng",
    if_exists="replace",
    progress_bar=None,
    table_schema=bq_schema,
)